In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# -*- coding: utf-8 -*-
"""
Updated ProtoNet with CutMix Augmentation on Query Set
"""

from google.colab import drive
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
import multiprocessing as mp
import os
import cv2
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from tqdm import tnrange

# Mount Drive
drive.mount('/content/gdrive')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def rand_bbox(height, width, lam):
    cut_rat = np.sqrt(1. - lam)
    cut_w = int(width * cut_rat)
    cut_h = int(height * cut_rat)
    cx = np.random.randint(width)
    cy = np.random.randint(height)
    bbx1 = np.clip(cx - cut_w // 2, 0, width)
    bby1 = np.clip(cy - cut_h // 2, 0, height)
    bbx2 = np.clip(cx + cut_w // 2, 0, width)
    bby2 = np.clip(cy + cut_h // 2, 0, height)
    return bbx1, bby1, bbx2, bby2

def apply_cutmix(images, labels, alpha=1.0):
    batch_size = images.size(0)
    indices = torch.randperm(batch_size)
    shuffled_images = images[indices]
    shuffled_labels = labels[indices]

    lam = np.random.beta(alpha, alpha)
    bbx1, bby1, bbx2, bby2 = rand_bbox(images.size(2), images.size(3), lam)
    images[:, :, bbx1:bbx2, bby1:bby2] = shuffled_images[:, :, bbx1:bbx2, bby1:bby2]
    lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (images.size(-1) * images.size(-2)))

    return images, labels, shuffled_labels, lam

def extract_sample(n_way, n_support, n_query, datax, datay, apply_cutmix_flag=False):
    sample = []
    query_labels = []
    K = np.random.choice(np.unique(datay), n_way, replace=False)

    for cls_idx, cls in enumerate(K):
        datax_cls = datax[datay == cls]
        perm = np.random.permutation(datax_cls)
        sample_cls = perm[:(n_support + n_query)]
        sample.append(sample_cls)
        query_labels.extend([cls_idx] * n_query)

    sample = np.array(sample)
    sample = torch.from_numpy(sample).float()
    sample = sample.permute(0, 1, 4, 2, 3)

    if apply_cutmix_flag:
        for i in range(n_way):
            support = sample[i, :n_support]
            query = sample[i, n_support:]
            labels = torch.tensor([i] * n_query)
            mixed_query, _, _, _ = apply_cutmix(query.clone(), labels)
            sample[i, n_support:] = mixed_query

    return {
        'images': sample,
        'n_way': n_way,
        'n_support': n_support,
        'n_query': n_query
    }

class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.size(0), -1)

def load_protonet_conv(x_dim, z_dim):
    def conv_block(in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=2, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

    encoder = nn.Sequential(
        conv_block(x_dim[0], 64),
        conv_block(64, 64),
        conv_block(64, 64),
        conv_block(64, z_dim)
    )

    for m in encoder.modules():
        if isinstance(m, nn.Conv2d):
            nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='relu')
            if m.bias is not None:
                nn.init.zeros_(m.bias)

    return ProtoNet(encoder)

class ProtoNet(nn.Module):
    def __init__(self, encoder):
        super().__init__()
        self.encoder = encoder.to(device)

    def set_forward_loss(self, sample, Max_Count_Act):
        sample_images = sample['images'].to(device)
        n_way = sample['n_way']
        n_support = sample['n_support']
        n_query = sample['n_query']

        x_support = sample_images[:, :n_support]
        x_query = sample_images[:, n_support:]

        target_inds = torch.arange(0, n_way).view(n_way, 1, 1).expand(n_way, n_query, 1).long()
        target_inds = Variable(target_inds, requires_grad=False).to(device)

        x = torch.cat([
            x_support.contiguous().view(n_way * n_support, *x_support.size()[2:]),
            x_query.contiguous().view(n_way * n_query, *x_query.size()[2:])
        ], 0)

        z = self.encoder(x)
        z_samples = z[:n_way * n_support]
        z_query = z[n_way * n_support:]

        z_proto = Concept_Prototype(z_samples, n_way, n_support, Max_Count_Act)
        dists = euclidean_dist(z_query, z_proto)
        log_p_y = F.log_softmax(-dists, dim=1).view(n_way, n_query, -1)
        loss_val = -log_p_y.gather(2, target_inds).squeeze().view(-1).mean()
        _, y_hat = log_p_y.max(2)
        acc_val = torch.eq(y_hat, target_inds.squeeze()).float().mean()

        return loss_val, {'loss': loss_val.item(), 'acc': acc_val.item(), 'y_hat': y_hat}

def Concept_Prototype(z_samples, n_way, n_support, Max_Count_Act):
    Prototypes = []
    for i in range(n_way):
        class_samples = z_samples[i * n_support:(i + 1) * n_support]
        Class_wise_prototypes = []
        for j in class_samples:
            avg_score = F.adaptive_avg_pool2d(j, output_size=1).flatten()
            max_score = F.adaptive_max_pool2d(j, output_size=1).flatten()
            var_score = torch.var(j.view(j.size(0), -1), dim=1)

            k_avg = Max_Count_Act // 3
            k_max = Max_Count_Act // 3
            k_var = Max_Count_Act - k_avg - k_max

            topk_avg = torch.argsort(avg_score, descending=True)[:k_avg]
            topk_max = torch.argsort(max_score, descending=True)[:k_max]
            topk_var = torch.argsort(var_score, descending=True)[:k_var]

            combined_indices = torch.unique(torch.cat((topk_avg, topk_max, topk_var)))
            if combined_indices.size(0) < Max_Count_Act:
                remaining = Max_Count_Act - combined_indices.size(0)
                additional = torch.argsort(avg_score, descending=True)
                for idx in additional:
                    if idx not in combined_indices:
                        combined_indices = torch.cat((combined_indices, idx.unsqueeze(0)))
                        if combined_indices.size(0) == Max_Count_Act:
                            break

            Filtered_feature_map = Mask_indices(j, combined_indices)
            Class_wise_prototypes.append(Filtered_feature_map)

        Class_wise_prototypes = torch.stack(Class_wise_prototypes)
        Class_wise_prototypes = torch.mean(Class_wise_prototypes, dim=0)
        Prototypes.append(Class_wise_prototypes)
    return torch.stack(Prototypes)

def Mask_indices(j, selected_indices):
    mask = torch.zeros_like(j)
    mask[selected_indices, :, :] = 1
    return j * mask

def euclidean_dist(x, y):
    n = x.size(0)
    m = y.size(0)
    d = x.size(1) * x.size(2) * x.size(3)
    x = x.view(n, 1, d).expand(n, m, d)
    y = y.view(1, m, d).expand(n, m, d)
    return torch.pow(x - y, 2).sum(2)

def train(model, optimizer, train_x, train_y, n_way, n_support, n_query, max_epoch, epoch_size, Max_Count_Act):
    scheduler = optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.5, last_epoch=-1)
    for epoch in range(max_epoch):
        running_loss, running_acc = 0.0, 0.0
        for episode in tnrange(epoch_size, desc="Epoch {:d} train".format(epoch + 1)):
            sample = extract_sample(n_way, n_support, n_query, train_x, train_y, apply_cutmix_flag=True)
            optimizer.zero_grad()
            loss, output = model.set_forward_loss(sample, Max_Count_Act)
            running_loss += output['loss']
            running_acc += output['acc']
            loss.backward()
            optimizer.step()
        scheduler.step()
        print(f"Epoch {epoch+1}: Loss {running_loss/epoch_size:.4f}, Acc {running_acc/epoch_size:.4f}")

def test(model, test_x, test_y, n_way, n_support, n_query, test_episode, Max_Count_Act):
    model.eval()
    total_acc = 0.0
    for _ in range(test_episode):
        sample = extract_sample(n_way, n_support, n_query, test_x, test_y)
        _, output = model.set_forward_loss(sample, Max_Count_Act)
        total_acc += output['acc']
    avg_acc = total_acc / test_episode
    print(f"Test Accuracy: {avg_acc:.4f}")
    return avg_acc
#-----------Main function-----------------------------------------------------

n_way = 2
n_support = 1
n_query = 5
Max_Count_Act=21 #21
epochs_to_run = [20,25,30,35,40]
epoch_size = 2000
test_episode = 600
#-------------------------set seeeds-----------------------------
np_seed = 0 # Set the seed for reproducibility
torch_seed = 0
Size=84
# Fix all seeds
np.random.seed(np_seed)
torch.manual_seed(torch_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(torch_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Load pickle files
PATH = '/content/gdrive/MyDrive/Refined-Pickle-Files/Refined-Pickle-Files/Pickle-'+str(Size)+'_'+str(Size)+'/'
Data='BC-100'

with open(PATH + Data + "_trainx.pkl", "rb") as f:
    trainx = pickle.load(f)
with open(PATH + Data + "_trainy.pkl", "rb") as f:
    trainy = pickle.load(f)
with open(PATH + Data + "_testx.pkl", "rb") as f:
    testx = pickle.load(f)
with open(PATH + Data + "_testy.pkl", "rb") as f:
    testy = pickle.load(f)

# Function to track the highest accuracy
def get_accuracy(model, testx, testy, n_way, n_support, n_query, test_episode,Max_Count_Act):
    # Assuming this function is available in your framework
    return test(model, testx, testy, n_way, n_support, n_query, test_episode,Max_Count_Act)

best_accuracy = 0
best_epoch = 0

# Training and testing for specified epochs
for epoch in epochs_to_run:
    # Initialize model and optimizer
    model = load_protonet_conv(x_dim=(3, Size, Size), z_dim=256)
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    print(f"-----------------------Training for {epoch} epochs-----------------------")
    train(model, optimizer, trainx, trainy, n_way, n_support, n_query, epoch, epoch_size,Max_Count_Act)

    print(f"-----------------------Testing after {epoch} epochs-----------------------")
    accuracy = get_accuracy(model, testx, testy, n_way, n_support, n_query, test_episode,Max_Count_Act)

    print(f"Accuracy after {epoch} epochs: {accuracy}")

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_epoch = epoch

print(f"\nHighest accuracy: {best_accuracy} achieved at epoch: {best_epoch}")


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
-----------------------Training for 20 epochs-----------------------


/tmp/ipython-input-2-2110954888.py:198: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for episode in tnrange(epoch_size, desc="Epoch {:d} train".format(epoch + 1)):


Epoch 1 train:   0%|          | 0/2000 [00:00<?, ?it/s]

Epoch 1: Loss 20.1316, Acc 0.5872


/tmp/ipython-input-2-2110954888.py:198: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for episode in tnrange(epoch_size, desc="Epoch {:d} train".format(epoch + 1)):


...

Epoch 18 train:   0%|          | 0/2000 [00:00<?, ?it/s]

Epoch 18: Loss 0.8382, Acc 0.7011


Epoch 19 train:   0%|          | 0/2000 [00:00<?, ?it/s]

Epoch 19: Loss 0.8030, Acc 0.6975


Epoch 20 train:   0%|          | 0/2000 [00:00<?, ?it/s]